In [1]:
import warnings
warnings.filterwarnings('ignore')

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop, SGD

from keras.datasets import mnist
import numpy as np

Using TensorFlow backend.


In [2]:
import scipy.io as spio

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

mat = spio.loadmat('oopen_smile2.mat', squeeze_me=True)

a=mat['OpenSmile']

X=a[:,0:118]

X = preprocessing.scale(X)
print(X.shape)
Y = mat['str_idx']
print(Y.shape)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(581595, 118)
(581595,)
(465276, 118)
(116319, 118)
(465276,)
(116319,)


In [3]:

num_classes = 2


In [4]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [5]:
timestep = 20

tmp_train = np.reshape(x_train[0], (1, len(x_train[0])))
print(tmp_train.shape)
tmp_train = np.repeat(tmp_train, timestep, axis=0)
print(tmp_train.shape)
print(len(tmp_train[0]))
print(len(tmp_train[:,:]))
ab = np.reshape(tmp_train, (1, len(tmp_train[:,:]), len(tmp_train[0])))
print(ab.shape)
print(x_train.shape[0])
x_train = np.repeat( ab, x_train.shape[0], axis=0)
print(x_train.shape)

(1, 118)
(20, 118)
118
20
(1, 20, 118)
465276
(465276, 20, 118)


In [6]:
timestep = 20

tmp_test = np.reshape(x_test[0], (1, len(x_test[0])))
print(tmp_test.shape)
tmp_test = np.repeat(tmp_test, timestep, axis=0)
print(tmp_test.shape)
print(len(tmp_test[0]))
print(len(tmp_test[:,:]))
abc = np.reshape(tmp_test, (1, len(tmp_test[:,:]), len(tmp_test[0])))
print(abc.shape)
x_test = np.repeat( abc, x_test.shape[0], axis=0)
print(x_test.shape)

(1, 118)
(20, 118)
118
20
(1, 20, 118)
(116319, 20, 118)


In [30]:
timestep = 20
from keras.layers import Flatten, LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(timestep, 118), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Activation('sigmoid'))
model.add(Dense(2))

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
#rms = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 20, 64)            46848     
_________________________________________________________________
dropout_19 (Dropout)         (None, 20, 64)            0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dropout_20 (Dropout)         (None, 32)                0         
_________________________________________________________________
activation_9 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 66        
Total params: 59,330
Trainable params: 59,330
Non-trainable params: 0
_________________________________________________________________


In [32]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
hist = model.fit(x_train, y_train, validation_split=0.2, verbose=1, nb_epoch=100, batch_size=32, shuffle=False, callbacks=[early_stopping])

Train on 372220 samples, validate on 93056 samples
Epoch 1/100
372220/372220 [==============================] - 224s 601us/step - loss: 8.0590 - acc: 0.0064 - val_loss: 8.0590 - val_acc: 0.0000e+00
Epoch 2/100
372220/372220 [==============================] - 223s 598us/step - loss: 8.0590 - acc: 0.0066 - val_loss: 8.0590 - val_acc: 0.0000e+00
Epoch 3/100
372220/372220 [==============================] - 205s 550us/step - loss: 8.0590 - acc: 0.0064 - val_loss: 8.0590 - val_acc: 0.0000e+00
Epoch 4/100
372220/372220 [==============================] - 205s 550us/step - loss: 8.0590 - acc: 0.0063 - val_loss: 8.0590 - val_acc: 0.0000e+00
Epoch 5/100
372220/372220 [==============================] - 204s 549us/step - loss: 8.0590 - acc: 0.0064 - val_loss: 8.0590 - val_acc: 0.0000e+00
Epoch 6/100
372220/372220 [==============================] - 204s 548us/step - loss: 8.0590 - acc: 0.0064 - val_loss: 8.0590 - val_acc: 0.0000e+00
Epoch 7/100
372220/372220 [==============================] - 204s 5

In [33]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 8.05904769897461
Test accuracy: 0.0
